# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks, code quality and readability (no single-letter variables, PEP8 compliant) etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [ ]:
# ! pip3 install pyspark

In [1]:
import findspark
findspark.init()


In [2]:
import pyspark
sc = pyspark.SparkContext(appName='jupyter')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-09-27 20:29:53,147 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [4]:
# Read the data
df = se.read.csv("hdfs:/data/clickstream.csv", header=True, sep="\t")

In [5]:
# Register a temporary table for SQL
df.createOrReplaceTempView("clickstream")

In [6]:
print(df.columns)

['user_id', 'session_id', 'event_type', 'event_page', 'timestamp']


### Spark SQL solution with <=2 queries

In [15]:
se.sql("""
    CREATE OR REPLACE TEMP VIEW ValidRoutes AS
    SELECT user_id, session_id, COLLECT_LIST(event_page) AS route
    FROM (
        SELECT user_id, session_id, event_page, timestamp
        FROM clickstream
        WHERE (user_id, session_id, timestamp) NOT IN (
            SELECT user_id, session_id, timestamp
            FROM clickstream
            WHERE event_type LIKE '%error%'
        )
        ORDER BY user_id, session_id, timestamp
    )
    GROUP BY user_id, session_id
""")


result_2q = se.sql("""
    SELECT CONCAT_WS('-', route) as route, COUNT(*) as count
    FROM ValidRoutes
    GROUP BY route
    ORDER BY count DESC
    LIMIT 30
""")

In [16]:
result_2q.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 2435|
|           main-main| 1459|
|      main-main-main|  850|
| main-main-main-main|  549|
|main-main-main-ma...|  321|
|main-main-main-ma...|  168|
|        main-archive|  153|
|         main-rabota|  145|
|       main-internet|  122|
|main-main-main-ma...|  118|
|           main-news|  102|
|          main-bonus|   99|
|  main-rabota-rabota|   92|
|    main-main-rabota|   86|
|   main-main-archive|   84|
|main-archive-archive|   84|
|        main-tariffs|   80|
|         main-online|   77|
|main-internet-int...|   73|
|     main-main-bonus|   73|
|          main-vklad|   72|
|      main-main-news|   68|
|main-main-main-ar...|   65|
|    main-bonus-bonus|   62|
|main-main-main-ma...|   58|
|    main-main-online|   55|
|      main-news-news|   53|
|main-bonus-bonus-...|   52|
|main-main-main-in...|   51|
|main-archive-arch...|   51|
+--------------------+-----+



In [ ]:
# result_2q.write.csv("hdfs:/data/result_sql_2q_Silaychev.csv", sep='\t')

### Spark SQL solution with 1 query

In [17]:
result_1q = se.sql("""
    WITH CleanSessions AS (
        SELECT user_id, session_id, event_page, timestamp
        FROM clickstream
        WHERE (user_id, session_id, timestamp) NOT IN (
            SELECT user_id, session_id, timestamp
            FROM clickstream
            WHERE event_type LIKE '%error%'
        )
        ORDER BY user_id, session_id, timestamp
    ),
    Routes AS (
        SELECT user_id, session_id, COLLECT_LIST(event_page) AS route
        FROM CleanSessions
        GROUP BY user_id, session_id
    )
    SELECT CONCAT_WS('-', route) as route, COUNT(*) as count
    FROM Routes
    GROUP BY route
    ORDER BY count DESC
    LIMIT 30
""")


### w/o duplicates
# result = spark.sql("""
#     WITH CleanSessions AS (
#         SELECT user_id, session_id, event_page, timestamp
#         FROM clickstream
#         WHERE (user_id, session_id, timestamp) NOT IN (
#             SELECT user_id, session_id, timestamp
#             FROM clickstream
#             WHERE event_type LIKE '%error%'
#         )
#         ORDER BY user_id, session_id, timestamp
#     ),
#     Routes AS (
#         SELECT user_id, session_id, ARRAY_DISTINCT(COLLECT_LIST(event_page)) AS route #  <-- in case we want to remove duplicates
#         FROM CleanSessions
#         GROUP BY user_id, session_id
#     )
#     SELECT CONCAT_WS('-', route) as route, COUNT(*) as count
#     FROM Routes
#     GROUP BY route
#     ORDER BY count DESC
#     LIMIT 30
# """)

In [19]:
result_1q.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 2435|
|           main-main| 1459|
|      main-main-main|  850|
| main-main-main-main|  549|
|main-main-main-ma...|  321|
|main-main-main-ma...|  168|
|        main-archive|  153|
|         main-rabota|  145|
|       main-internet|  122|
|main-main-main-ma...|  118|
|           main-news|  102|
|          main-bonus|   99|
|  main-rabota-rabota|   92|
|    main-main-rabota|   86|
|   main-main-archive|   84|
|main-archive-archive|   84|
|        main-tariffs|   80|
|         main-online|   77|
|main-internet-int...|   73|
|     main-main-bonus|   73|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# result_1q.write.csv("hdfs:/data/result_sql_1q_Silaychev.csv", sep='\t')

#### Complexity evaluation for Spark SQL solutions.
- Solution with 1 query:

1. Filtering corrupted sessions (using the NOT IN clause): This operation's complexity is *O(N)*, where N is the number of rows in the dataset.

2. Grouping by user_id and session_id and collecting lists: This is also a *O(N)* operation, as we're iterating through each row in the dataset.

3. Aggregating based on the route and counting: In the worst-case scenario, this can be O(N) if every session has a unique route.

Overall, the SQL solution is *O(N)*.

- Solution with 2 queries:

The complexity remains *O(N)* for similar reasons as mentioned in the 1-query solution. Breaking the solution into two queries doesn't change the overall complexity.

### Spark RDD solution

In [4]:
# Reading and setting up the RDD
rdd = sc.textFile("hdfs:/data/clickstream.csv")

In [5]:
header = rdd.first()

In [10]:
rdd = rdd.filter(lambda line: line != header)

In [11]:
# Split the rows based on the comma
rdd_split = rdd.map(lambda line: line.split("\t"))

In [12]:
# Helper function to identify rows with errors
def has_error(row):
    return 'error' in row[2]

In [13]:
# Filter sessions with errors
filtered_rdd = rdd_split.filter(lambda row: not has_error(row))

In [14]:
# Extract and build routes
routes = filtered_rdd.map(lambda row: ((row[0], row[1]), [row[3]])).reduceByKey(lambda a, b: a + b).map(lambda row: ("-".join(row[1]), 1))

In [15]:
# Count routes
route_counts = routes.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False)

In [ ]:
### IN CASE OF DISTINCT ROUTES --- to avoid duplicates

# ... [Initial setup remains the same]

# # Extract and build distinct routes
# routes = filtered_rdd.map(lambda row: ((row[0], row[1]), [row[3]])).reduceByKey(lambda a, b: list(set(a + b))).map(lambda row: ("-".join(row[1]), 1))

# # Count routes
# route_counts = routes.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False)



In [ ]:
# route_counts.take(30).saveAsTextFile("hdfs:/data/result_rdd_Silaychev.csv")

In [16]:
# Show top 30 results
route_counts.take(30)

[('main', 2435),
 ('main-main', 1459),
 ('main-main-main', 850),
 ('main-main-main-main', 549),
 ('main-main-main-main-main', 321),
 ('main-main-main-main-main-main', 168),
 ('main-archive', 153),
 ('main-rabota', 145),
 ('main-internet', 122),
 ('main-main-main-main-main-main-main', 118),
 ('main-news', 102),
 ('main-bonus', 99),
 ('main-rabota-rabota', 92),
 ('main-main-rabota', 86),
 ('main-archive-archive', 84),
 ('main-main-archive', 84),
 ('main-tariffs', 80),
 ('main-online', 76),
 ('main-main-bonus', 74),
 ('main-internet-internet', 73),
 ('main-vklad', 72),
 ('main-main-news', 68),
 ('main-main-main-archive', 65),
 ('main-bonus-bonus', 62),
 ('main-main-main-main-main-main-main-main', 58),
 ('main-main-online', 55),
 ('main-news-news', 53),
 ('main-bonus-bonus-bonus', 52),
 ('main-archive-archive-archive', 51),
 ('main-main-main-internet', 51)]

#### Complexity evaluation for Spark RDD solution


1. Filtering rows: Complexity is *O(N)*.
2. Building routes using reduceByKey: Complexity is *O(N)*. This is because reduceByKey performs a distributed aggregation.
3. Counting routes using reduceByKey: Again, this is *O(N)* in the worst-case scenario if each route is unique.
4. Sorting routes based on count: Sorting complexity is *O(N log N)*.
5. Considering all these steps, the RDD solution has a complexity of *O(N log N)* due to the sorting step.

### Spark DF Solution

In [4]:
from pyspark.sql import functions as F

In [5]:
# Load the clickstream.csv data into a DataFrame
df = se.read.csv("hdfs:/data/clickstream.csv", header=True, sep="\t", inferSchema=True)

In [6]:
# Remove corrupted sessions
sessions_without_errors = df.withColumn("has_error", F.col("event_type").like("%error%")).filter(~F.col("has_error"))


In [7]:
# Create routes
routes_df = sessions_without_errors.groupBy("user_id", "session_id").agg(F.collect_list("event_page").alias("route"))


In [8]:
# Count the routes
result_df = routes_df.groupBy("route").count().orderBy(F.desc("count")).limit(30)


In [ ]:
### To avoid duplicates


### ... [Initial setup remains the same]

# # Create routes with distinct event pages
# distinct_routes_df = sessions_without_errors.groupBy("user_id", "session_id").agg(F.array_distinct(F.collect_list("event_page")).alias("route"))

# # Count the distinct routes
# result_df = distinct_routes_df.groupBy("route").count().orderBy(F.desc("count")).limit(30)


In [ ]:
# result_df.write.csv("hdfs:/data/result_df_Silaychev.csv", sep='\t')

In [10]:
# Save results
result_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|              [main]| 2435|
|        [main, main]| 1459|
|  [main, main, main]|  850|
|[main, main, main...|  549|
|[main, main, main...|  321|
|[main, main, main...|  168|
|     [main, archive]|  152|
|      [main, rabota]|  144|
|    [main, internet]|  122|
|[main, main, main...|  118|
|        [main, news]|  102|
|       [main, bonus]|   99|
|[main, rabota, ra...|   92|
|[main, main, rabota]|   86|
|[main, main, arch...|   84|
|[main, archive, a...|   84|
|     [main, tariffs]|   80|
|      [main, online]|   76|
| [main, main, bonus]|   74|
|[main, internet, ...|   73|
|       [main, vklad]|   72|
|  [main, main, news]|   68|
|[main, main, main...|   64|
|[main, bonus, bonus]|   62|
|[main, main, main...|   58|
|[main, main, online]|   55|
|  [main, news, news]|   53|
|[main, bonus, bon...|   51|
|[main, main, main...|   51|
|[main, archive, a...|   50|
+--------------------+-----+



#### Complexity evaluation for Spark DF solution


1. Filtering corrupted sessions: O(N) complexity.
2. Grouping and creating routes: O(N) complexity.
3. Aggregating based on the route and counting: O(N) complexity.

The Spark DataFrame solution has a complexity of O(N).

When analyzing user behavior through clickstreams or similar logging mechanisms, slight differences in outputs can arise due to various reasons, even if the core logic of the solution remains consistent. 

Here are a few reasons why outputs might have slight differences, that I came up with:

- Non-deterministic Aggregation: Some operations, especially when parallelized over many nodes in a distributed system like Spark, might not always return results in the same exact order. For example, when you're collecting pages into a list, the order might differ between runs. However, when I checked, we had only one node. Thus, this reason is not applicable.

- Error Handling: Different approaches might handle errors differently. For example, if a user session has an error event, one approach might discard the entire session while another might only discard the events after the error. I tried to ensure the consistency, however it is hard to have case-by-case view on the matter.

- Intermediate Steps: Even if two approaches aim to achieve the same final result, they might have different intermediate steps that can lead to slight differences in the final output.